---
Distributed Computing
---

![](images/cloud-cartoon.jpg)

What is distributed computing?
----

__Many computers with shared computing goal__:
![](images/clientserver.png)

[Additional Slides](http://wla.berkeley.edu/~cs61a/fa11/61a-python/content/www/slides/61A-031-Distributed_1pps.pdf)

---
Why distrubted computing?
----

Massive gains in performance and effiency. Horizontal scaling!

__Downsides__:
- Coordination overhead
- Harder to reason about and debug

---
Characteristics of distributed systems
---
1. Independent computers
2. (Often) In different locations
3. Connected by a network
4. Communicate by passing messages to each other
5. A shared computational goal

![](images/architecture.png)

Other architectures are possible (e.g., peer-to-peer)

SPOF = __Single Point of Failure__

---
Why is distrubted computing hard?
---

Come on, I just put a bunch computers together on a local network. 

__Fallacies of distributed computing:__

- The network is reliable.
- Latency is zero.
- Bandwidth is infinite.
- The network is secure.
- Topology doesn’t change.
- There is one administrator.
- Transport cost is zero.
- The network is homogeneous.

[Source](https://en.wikipedia.org/wiki/Fallacies_of_distributed_computing)

![](images/messages.png)

This means there will be __Logs, Logs, Logs__

---
How Data Scientists can use Distributed Computing?
---

Motivating Story
----

![](images/cluster_speedup.png)

When doing model evaluations and parameters tuning, many models must be trained independently on the same data. This is an embarrassingly parallel problem but having a copy of the dataset in memory for each process is waste of RAM:

<img src="https://s3-us-west-2.amazonaws.com/dsci6007/assets/grid_search_parameters.png" style="display:inline; width: 49%" />
<img src="https://s3-us-west-2.amazonaws.com/dsci6007/assets/grid_search_cv_splits.png" style="display:inline; width: 49%" />

### Parallel Model Selection and Assessment
When doing 3 folds cross validation on a 9 parameters grid, a naive implementation could read the data from the disk and load it in memory 27 times. If this happens concurrently (e.g. on a compute node with 32 cores) the RAM might blow up hence breaking the potential linear speed up.

### Check for understanding
<details><summary>How many copies would be needed to do 5-fold cross-validation if we added `1000` as a possibility for `param_1`?</summary>
60</details>


### Check for understanding

<details><summary>What does it mean for a task to be "embarrassingly parallel"?</summary>
Little or no effort is required to separate the problem into a number of parallel tasks.</details>

<details><summary>List some algorithms that are and are not embarrassingly parallel</summary>
<table><tr><th>Embarassingly Parallel<th>Not Embarassingly Parallel</th></tr>
<tr><td>Random Forests</td><td>AdaBoost</td></tr>
<tr><td>Grid Search</td><td>MaxEnt</td></tr>
<tr><td>...</td><td>...</td></tr>
</table></details>

How Jupyter works
----

![](http://ipython.readthedocs.org/en/stable/_images/ipy_kernel_and_terminal.png)

It is a Unix Big Data System!

---
`ipyparallel`, a Primer
---

*Adapted from [Olivier Grisel](http://ogrisel.com/)'s [Parallel Machine Learning Tutorial](https://github.com/ogrisel/parallel_ml_tutorial) on [Distributed Model Selection and Assessment](https://github.com/ogrisel/parallel_ml_tutorial/blob/master/notebooks/06%20-%20Distributed%20Model%20Selection%20and%20Assessment.ipynb)*

This section gives a primer on some tools best utilizing computational resources when doing predictive modeling in the Python / NumPy ecosystem namely:

- optimal usage of available CPUs and cluster nodes with **`ipyparallel`**

- optimal memory re-use using shared memory between Python processes using **`numpy.memmap`** and **`joblib`**

### What is so great about `ipyparallel`:

- Single node multi-CPUs
- Multiple node multi-CPUs
- Interactive In-memory computing
- Jupyter integration with `%px` and `%%px` magics
- Possibility to interactively connect to individual computing processes to launch interactive debugger (`#priceless`)



---
Installing IPython for parallel computing
---

[repo](https://github.com/ipython/ipyparallel)

[Read the docs](http://ipyparallel.readthedocs.org/en/latest/)

Architecture overview
---

![](http://ipyparallel.readthedocs.org/en/latest/_images/wideView.png)

[Source](http://ipyparallel.readthedocs.org/en/latest/intro.html#architecture-overview)

### Let's get started:

Let start an IPython cluster using the `ipcluster` common (usually run from your operating system console). To make sure that we are not running several clusters on the same host, let's try to shut down any running IPython cluster first:

In [1]:
!ipcluster stop

2016-06-06 09:31:36.212 [IPClusterStop] CRITICAL | Could not read pid file, cluster is probably not running.


In [1]:
!ipcluster start -n=4 --daemon

Optional: Go to the "Cluster" tab of the notebook and **start a local cluster with 2 engines**. Then come back here. 

We should now be able to use our cluster from our notebook session (or any other Python process running on localhost):

In [3]:
from ipyparallel import Client

client = Client()

len(client)

4

#### The %px and %%px magics

All the engines of the client can be accessed imperatively using the `%px` and `%%px` IPython cell magics:

[Docs](http://ipyparallel.readthedocs.io/en/latest/magics.html)

In [4]:
%%px

import os
import socket

print("This is running in process with pid {0} on host '{1}'.".format(
      os.getpid(), socket.gethostname()))

[stdout:0] This is running in process with pid 77599 on host 'Alessandros-MacBook-Pro.local'.
[stdout:1] This is running in process with pid 77601 on host 'Alessandros-MacBook-Pro.local'.
[stdout:2] This is running in process with pid 77600 on host 'Alessandros-MacBook-Pro.local'.
[stdout:3] This is running in process with pid 77602 on host 'Alessandros-MacBook-Pro.local'.


The content of the `__main__` namespace can also be read and written via the `%px` magic:

In [5]:
%px a = 1

In [6]:
%px print(a)

[stdout:0] 1
[stdout:1] 1
[stdout:2] 1
[stdout:3] 1


In [7]:
%%px

a *= 2
print(a)

[stdout:0] 2
[stdout:1] 2
[stdout:2] 2
[stdout:3] 2


It is possible to restrict the `%px` and `%%px` magic instructions to specific engines:

In [8]:
%%px --targets=-1
a *= 2
print(a)

4


In [9]:
%px print(a)

[stdout:0] 2
[stdout:1] 2
[stdout:2] 2
[stdout:3] 4


#### The DirectView objects

Cell magics are very nice to work interactively from the notebook but it's also possible to replicate their behavior programmatically with more flexibility with a `DirectView` instance. A `DirectView` can be created by slicing the client object:

In [10]:
all_engines = client[:]
all_engines

<DirectView [0, 1, 2, 3]>

The namespace of the `__main__` module of each running python engine can be accessed in read and write mode as a python dictionary:

In [11]:
all_engines['ab'] = 13

In [12]:
all_engines['ab']

[13, 13, 13, 13]

In [13]:
%%px
print(ab)

[stdout:0] 13
[stdout:1] 13
[stdout:2] 13
[stdout:3] 13


Direct views can also execute the same code in parallel on each engine of the view:

In [14]:
def my_sum(a, b):
    return a + b

my_sum_apply_results = all_engines.apply(my_sum, 11, 31)
my_sum_apply_results

<AsyncResult: my_sum>

The ouput of the `apply` method is an asynchronous handle returned immediately without waiting for the end of the computation. To block until the results are ready use:

In [15]:
my_sum_apply_results.get()

[42, 42, 42, 42]

Here is a more useful example to fetch the network hostname of each engine in the cluster. Let's study it in more details:

In [16]:
def hostname():
    """Return the name of the host where the function is being called"""
    import socket
    return socket.gethostname()

hostname_apply_result = all_engines.apply(hostname)

When doing the above, the `hostname` function is first defined locally (the client python process). The `DirectView.apply` method introspects it, serializes its name and bytecode and ships it to each engine of the cluster where it is reconstructed as local function on each engine. This function is then called on each engine of the view with the optionally provided arguments.

In return, the client gets a python object that serves as an handle to asynchronously fetch the list of the results of the calls:

In [17]:
hostname_apply_result

<AsyncResult: hostname:finished>

In [18]:
hostname_apply_result.get()

['Alessandros-MacBook-Pro.local',
 'Alessandros-MacBook-Pro.local',
 'Alessandros-MacBook-Pro.local',
 'Alessandros-MacBook-Pro.local']

It is also possible to key the results explicitly with the engine ids with the `AsyncResult.get_dict` method. This is a very simple idiom to fetch metadata on the runtime environment of each engine of the direct view:

In [19]:
hostnames = hostname_apply_result.get_dict()
hostnames

{0: 'Alessandros-MacBook-Pro.local',
 1: 'Alessandros-MacBook-Pro.local',
 2: 'Alessandros-MacBook-Pro.local',
 3: 'Alessandros-MacBook-Pro.local'}

It can be handy to invert this mapping to find one engine id per host in the cluster so as to execute host specific operation:

In [20]:
one_engine_by_host = dict((hostname, engine_id) for engine_id, hostname
                      in hostnames.items())
one_engine_by_host

{'Alessandros-MacBook-Pro.local': 3}

In [21]:
one_engine_per_host_view = client[one_engine_by_host.values()]
one_engine_per_host_view

<DirectView [3]>

**Trick:** you can even use those engines ids to execute shell commands in parallel on each host of the cluster:

```python
%%px --targets=one_engine_by_host.values()

!pip install scikit-learn
```

In [22]:
one_engine_by_host.values()

[3]

#### Note on Importing Modules on Remote Engines

In the previous example we put the `import socket` statement inside the body of the `hostname` function to make sure to make sure that is is available when the rest of the function is executed in the python processes of the remote engines.

Alternatively it is possible to import the required modules ahead of time on all the engines of a directview using a context manager / with syntax:

In [23]:
with all_engines.sync_imports():
    import numpy

importing numpy on engine(s)


However this method does **not** support alternative import syntaxes:
    
    >>> import numpy as np
    >>> from numpy import linalg

Hence the method of importing in the body of the "applied" functions is more flexible. Additionally, this does not pollute the `__main__` namespace of the engines as it only impact the local namespace of the function itself.

---
**Optional Exercise**:
---

- Write a function that returns the amemory usage of each engine process in the cluster.
- Allocate a largish numpy array of zeros of known size (e.g. 100MB) on each engine of the cluster.

Hints:

Use the `psutil` module to collect the runtime info on a specific process or host. For instance to fetch the memory usage of the currently running process in MB:

    >>> import os
    >>> import psutil
    >>> psutil.Process(os.getpid()).get_memory_info().rss / 1e6

To allocate a numpy array with 1000 zeros stored as 64bit floats you can use:

    >>> import numpy as np
    >>> z = np.zeros(1000, dtype=np.float64)

The size in bytes of such a numpy array can then be fetched with ``z.nbytes``:
    
    >>> z.nbytes / 1e6
    0.008

#### Load Balanced View

`LoadBalancedView` is an alternative to the `DirectView` to run one function call at a time on a free engine.

In [26]:
lv = client.load_balanced_view()

In [27]:
def slow_square(x):
    import time
    time.sleep(2)
    return x ** 2

In [28]:
result = lv.apply(slow_square, 4)

In [29]:
result

<AsyncResult: slow_square>

In [30]:
result.ready()

False

In [31]:
result.get()  # blocking call

16

It is possible to spread some tasks among the engines of the LB view by passing a callable and an iterable of task arguments to the `LoadBalancedView.map` method:

In [30]:
results = lv.map(slow_square, [0, 1, 2, 3])
results

<AsyncMapResult: slow_square>

In [31]:
results.ready()

False

In [32]:
results.progress

0

In [33]:
# results.abort()

In [34]:
# Iteration on AsyncMapResult is blocking
for r in results:
    print(r)

0
1
4
9


The load balanced view will be used in the following to schedule work on the cluster while being able to monitor progress and occasionally add new computing nodes to the cluster while computing to speed up the processing when using EC2 and StarCluster (see later).

### Pop Quiz
<details><summary>What is the difference between a direct view and a load balanced view?</summary>
In a direct view, engines are addressed explicitly.<br />
In a load balanced view, the scheduler is trusted with assigning work to appropriate engines.</details>


## Sharing Read-only Data between Processes on the Same Host with Memmapping

The numpy package makes it possible to memory map large contiguous chunks of binary files as shared memory for all the Python processes running on a given host:

In [35]:
%px import numpy as np

Creating a `numpy.memmap` instance with the `w+` mode creates a file on the filesystem and zeros its content. Let's do it from the first engine process or our current IPython cluster:

In [36]:
%%px --targets=-1  
# replace with one_engine_by_host.values()

# Cleanup any existing file from past session (necessary for windows)
import os
if os.path.exists('small.mmap'):
    os.unlink('small.mmap')

mm_w = np.memmap('small.mmap', shape=10, dtype=np.float32, mode='w+')
print(mm_w)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


This binary file can then be mapped as a new numpy array by all the engines having access to the same filesystem. The `mode='r+'` opens this shared memory area in read write mode:

In [37]:
%%px

mm_r = np.memmap('small.mmap', dtype=np.float32, mode='r+')
print(mm_r)

[stdout:0] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:1] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:2] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:3] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [38]:
%%px --targets=-1  
# replace with one_engine_by_host.values()

mm_w[0] = 42
print(mm_w)
print(mm_r)

[ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
[ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [39]:
%px print(mm_r)

[stdout:0] [ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
[stdout:1] [ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
[stdout:2] [ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
[stdout:3] [ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


Memory mapped arrays created with `mode='r+'` can be modified and the modifications are shared with all the engines:

In [40]:
%%px --targets=1

mm_r[1] = 43

In [41]:
%%px
print(mm_r)

[stdout:0] [ 42.  43.   0.   0.   0.   0.   0.   0.   0.   0.]
[stdout:1] [ 42.  43.   0.   0.   0.   0.   0.   0.   0.   0.]
[stdout:2] [ 42.  43.   0.   0.   0.   0.   0.   0.   0.   0.]
[stdout:3] [ 42.  43.   0.   0.   0.   0.   0.   0.   0.   0.]


Be careful those, there is no builtin read nor write lock available on this such datastructures so it's better to avoid concurrent read & write operations on the same array segments unless there engine operations are made to cooperate with some synchronization or scheduling orchestrator.

Memmap arrays generally behave very much like regular in-memory numpy arrays:

In [42]:
%%px
print("sum={0:.3}, mean={1:.3}, std={2:.3}".format(
    mm_r.sum(), np.mean(mm_r), np.std(mm_r)))

[stdout:0] sum=85., mean=8.5, std=17.
[stdout:1] sum=85., mean=8.5, std=17.
[stdout:2] sum=85., mean=8.5, std=17.
[stdout:3] sum=85., mean=8.5, std=17.


Before allocating more data in memory on the cluster let us define a couple of utility functions from the previous exercise (and more) to monitor what is used by which engine and what is still free on the cluster as a whole:

In [43]:
def get_host_free_memory(client):
    """Free memory on each host of the cluster in MB."""
    all_engines = client[:]
    def hostname():
        import socket
        return socket.gethostname()
    
    hostnames = all_engines.apply(hostname).get_dict()
    one_engine_per_host = dict((hostname, engine_id)
                               for engine_id, hostname
                               in hostnames.items())

    def host_free_memory():
        import psutil
        return psutil.virtual_memory().free / 1e6
    
    
    host_mem = client[one_engine_per_host.values()].apply(
        host_free_memory).get_dict()
    
    return dict((hostnames[eid], m) for eid, m in host_mem.items())

In [44]:
get_host_free_memory(client)

{'Alessandros-MacBook-Pro.local': 1522.712576}

Let's allocate a 80MB memmap array in the first engine and load it in readwrite mode in all the engines:

In [45]:
%%px --targets=-1  
# replace with one_engine_by_host.values()

# Cleanup any existing file from past session (necessary for windows)
import os
if os.path.exists('big.mmap'):
    os.unlink('big.mmap')

np.memmap('big.mmap', shape=10 * int(1e6), dtype=np.float64, mode='w+')

Out[3:14]: memmap([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [46]:
%%px
ls -lh big.mmap

[stdout:0] -rw-r--r--  1 alessandro  staff    76M Jun  6 09:39 big.mmap
[stdout:1] -rw-r--r--  1 alessandro  staff    76M Jun  6 09:39 big.mmap
[stdout:2] -rw-r--r--  1 alessandro  staff    76M Jun  6 09:39 big.mmap
[stdout:3] -rw-r--r--  1 alessandro  staff    76M Jun  6 09:39 big.mmap


In [47]:
get_host_free_memory(client)

{'Alessandros-MacBook-Pro.local': 1522.93376}

No significant memory was used in this operation as we just asked the OS to allocate the buffer on the hard drive and just maitain a virtual memory area as a cheap reference to this buffer.

Let's open new references to the same buffer from all the engines at once:

In [48]:
%px %time big_mmap = np.memmap('big.mmap', dtype=np.float64, mode='r+')

[stdout:0] 
CPU times: user 249 µs, sys: 750 µs, total: 999 µs
Wall time: 10.8 ms
[stdout:1] 
CPU times: user 217 µs, sys: 318 µs, total: 535 µs
Wall time: 25.5 ms
[stdout:2] 
CPU times: user 513 µs, sys: 1.04 ms, total: 1.55 ms
Wall time: 24.3 ms
[stdout:3] 
CPU times: user 278 µs, sys: 1.3 ms, total: 1.58 ms
Wall time: 19.2 ms


In [49]:
%px big_mmap

Out[0:13]: memmap([ 0.,  0.,  0., ...,  0.,  0.,  0.])

Out[1:14]: memmap([ 0.,  0.,  0., ...,  0.,  0.,  0.])

Out[2:13]: memmap([ 0.,  0.,  0., ...,  0.,  0.,  0.])

Out[3:17]: memmap([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [50]:
get_host_free_memory(client)

{'Alessandros-MacBook-Pro.local': 1520.328704}

No physical memory was allocated in the operation as it just took a couple of ms to do so. This is also confirmed by the engines process stats:

Let's trigger an actual load of the data from the drive into the in-memory disk cache of the OS, this can take some time depending on the speed of the hard drive (on the order of 100MB/s to 300MB/s hence 3s to 8s for this dataset):

In [51]:
%%px --targets=-1  
# replace with one_engine_by_host.values()

%time np.sum(big_mmap)

CPU times: user 18.8 ms, sys: 43.4 ms, total: 62.2 ms
Wall time: 202 ms


Out[3:18]: memmap(0.0)

In [52]:
get_host_free_memory(client)

{'Alessandros-MacBook-Pro.local': 1418.514432}

We can see that the first engine has now access to the data in memory and the free memory on the host has decreased by the same amount.

We can now access this data from all the engines at once much faster as the disk will no longer be used: the shared memory buffer will instead accessed directly by all the engines:

In [53]:
%px %time np.sum(big_mmap)

[stdout:0] 
CPU times: user 24.7 ms, sys: 21.1 ms, total: 45.8 ms
Wall time: 61.5 ms
[stdout:1] 
CPU times: user 25.2 ms, sys: 21.2 ms, total: 46.4 ms
Wall time: 63.7 ms
[stdout:2] 
CPU times: user 25 ms, sys: 21 ms, total: 46.1 ms
Wall time: 70 ms
[stdout:3] 
CPU times: user 15.8 ms, sys: 95 µs, total: 15.9 ms
Wall time: 19.2 ms


Out[0:14]: memmap(0.0)

Out[1:15]: memmap(0.0)

Out[2:14]: memmap(0.0)

Out[3:19]: memmap(0.0)

In [54]:
get_host_free_memory(client)

{'Alessandros-MacBook-Pro.local': 1426.706432}

So it seems that the engines have loaded a whole copy of the data but this actually not the case as the total amount of free memory was not impacted by the parallel access to the shared buffer. Furthermore, once the data has been preloaded from the hard drive using one process, all the of the other processes on the same host can access it almost instantly saving a lot of IO wait.

This strategy makes it very interesting to load the readonly datasets of machine learning problems, especially when the same data is reused over and over by concurrent processes as can be the case when doing learning curves analysis or grid search.

## Memmaping Nested Numpy-based Data Structures with Joblib

joblib is a utility library included in the sklearn package. Among other things it provides tools to serialize objects that comprise large numpy arrays and reload them as memmap backed datastructures.

To demonstrate it, let's create an arbitrary python datastructure involving numpy arrays:

In [55]:
import numpy as np

class MyDataStructure(object):
    
    def __init__(self, shape):
        self.float_zeros = np.zeros(shape, dtype=np.float32)
        self.integer_ones = np.ones(shape, dtype=np.int64)
        
data_structure = MyDataStructure((3, 4))
data_structure.float_zeros, data_structure.integer_ones

(array([[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]], dtype=float32), array([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]]))

We can now persist this datastructure to disk:

In [56]:
from sklearn.externals import joblib

joblib.dump(data_structure, 'data_structure.pkl')

['data_structure.pkl',
 'data_structure.pkl_01.npy',
 'data_structure.pkl_02.npy']

In [57]:
!ls -l data_structure*

-rw-r--r--  1 alessandro  staff  273 Jun  6 09:50 data_structure.pkl
-rw-r--r--  1 alessandro  staff  128 Jun  6 09:50 data_structure.pkl_01.npy
-rw-r--r--  1 alessandro  staff  176 Jun  6 09:50 data_structure.pkl_02.npy


A memmapped copy of this datastructure can then be loaded:

In [58]:
memmaped_data_structure = joblib.load('data_structure.pkl', mmap_mode='r+')
memmaped_data_structure.float_zeros, memmaped_data_structure.integer_ones

(memmap([[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]], dtype=float32), memmap([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]]))

## Memmaping CV Splits for Multiprocess Dataset Sharing

We can leverage the previous tools to build a utility function that extracts Cross Validation splits ahead of time to persist them on the hard drive in a format suitable for memmaping by IPython engine processes.

In [59]:
from sklearn.externals import joblib
from sklearn.cross_validation import ShuffleSplit
import os

def persist_cv_splits(X, y, n_cv_iter=5, name='data',
    suffix="_cv_%03d.pkl", test_size=0.25, random_state=None):
    """Materialize randomized train test splits of a dataset."""

    cv = ShuffleSplit(X.shape[0], n_iter=n_cv_iter,
        test_size=test_size, random_state=random_state)
    cv_split_filenames = []
    
    for i, (train, test) in enumerate(cv):
        cv_fold = (X[train], y[train], X[test], y[test])
        cv_split_filename = name + suffix % i
        cv_split_filename = os.path.abspath(cv_split_filename)
        joblib.dump(cv_fold, cv_split_filename)
        cv_split_filenames.append(cv_split_filename)
    
    return cv_split_filenames

Let's try it on the digits dataset, we can run this from the :

In [60]:
from sklearn.datasets import load_digits

digits = load_digits()
digits_split_filenames = persist_cv_splits(digits.data, digits.target,
    name='digits', random_state=42)
digits_split_filenames

['/Users/alessandro/zipfian/DSCI6007-instructor/week3/3_1_distrubted_systems/lecture/digits_cv_000.pkl',
 '/Users/alessandro/zipfian/DSCI6007-instructor/week3/3_1_distrubted_systems/lecture/digits_cv_001.pkl',
 '/Users/alessandro/zipfian/DSCI6007-instructor/week3/3_1_distrubted_systems/lecture/digits_cv_002.pkl',
 '/Users/alessandro/zipfian/DSCI6007-instructor/week3/3_1_distrubted_systems/lecture/digits_cv_003.pkl',
 '/Users/alessandro/zipfian/DSCI6007-instructor/week3/3_1_distrubted_systems/lecture/digits_cv_004.pkl']

In [61]:
ls -lh digits*

-rw-r--r--  1 alessandro  staff   304B Jun  6 09:50 digits_cv_000.pkl
-rw-r--r--  1 alessandro  staff   674K Jun  6 09:50 digits_cv_000.pkl_01.npy
-rw-r--r--  1 alessandro  staff    11K Jun  6 09:50 digits_cv_000.pkl_02.npy
-rw-r--r--  1 alessandro  staff   225K Jun  6 09:50 digits_cv_000.pkl_03.npy
-rw-r--r--  1 alessandro  staff   3.6K Jun  6 09:50 digits_cv_000.pkl_04.npy
-rw-r--r--  1 alessandro  staff   304B Jun  6 09:50 digits_cv_001.pkl
-rw-r--r--  1 alessandro  staff   674K Jun  6 09:50 digits_cv_001.pkl_01.npy
-rw-r--r--  1 alessandro  staff    11K Jun  6 09:50 digits_cv_001.pkl_02.npy
-rw-r--r--  1 alessandro  staff   225K Jun  6 09:50 digits_cv_001.pkl_03.npy
-rw-r--r--  1 alessandro  staff   3.6K Jun  6 09:50 digits_cv_001.pkl_04.npy
-rw-r--r--  1 alessandro  staff   304B Jun  6 09:50 digits_cv_002.pkl
-rw-r--r--  1 alessandro  staff   674K Jun  6 09:50 digits_cv_002.pkl_01.npy
-rw-r--r--  1 alessandro  staff    11K Jun  6 09:50 digits_cv_002.pkl_02.npy
-rw-r--r--  1 alessa

Each of the persisted CV splits can then be loaded back again using memmaping:

In [62]:
X_train, y_train, X_test, y_test = joblib.load(
    'digits_cv_002.pkl',  mmap_mode='r+')

In [63]:
X_train

memmap([[  0.,   1.,  13., ...,   1.,   0.,   0.],
       [  0.,   0.,   7., ...,   9.,   0.,   0.],
       [  0.,   0.,   0., ...,  13.,   1.,   0.],
       ..., 
       [  0.,   0.,   4., ...,  16.,   1.,   0.],
       [  0.,   0.,   2., ...,  15.,   8.,   0.],
       [  0.,   0.,   0., ...,   3.,   0.,   0.]])

In [64]:
y_train

memmap([5, 3, 1, ..., 8, 6, 4])

---
Parallel Model Selection and Grid Search
---

Let's leverage IPython.parallel and the Memory Mapping features of joblib to write a custom grid search utility that runs on cluster in a memory efficient manner.

Assume that we want to reproduce the grid search from the previous session:

In [65]:
import numpy as np
from pprint import pprint

svc_params = {
    'C': np.logspace(-1, 2, 4),
    'gamma': np.logspace(-4, 0, 5),
}
pprint(svc_params)

{'C': array([   0.1,    1. ,   10. ,  100. ]),
 'gamma': array([  1.00000000e-04,   1.00000000e-03,   1.00000000e-02,
         1.00000000e-01,   1.00000000e+00])}


`GridSearchCV` internally uses the following `ParameterGrid` utility iterator class to build the possible combinations of parameters:

In [66]:
from sklearn.grid_search import ParameterGrid

list(ParameterGrid(svc_params))

[{'C': 0.10000000000000001, 'gamma': 0.0001},
 {'C': 0.10000000000000001, 'gamma': 0.001},
 {'C': 0.10000000000000001, 'gamma': 0.01},
 {'C': 0.10000000000000001, 'gamma': 0.10000000000000001},
 {'C': 0.10000000000000001, 'gamma': 1.0},
 {'C': 1.0, 'gamma': 0.0001},
 {'C': 1.0, 'gamma': 0.001},
 {'C': 1.0, 'gamma': 0.01},
 {'C': 1.0, 'gamma': 0.10000000000000001},
 {'C': 1.0, 'gamma': 1.0},
 {'C': 10.0, 'gamma': 0.0001},
 {'C': 10.0, 'gamma': 0.001},
 {'C': 10.0, 'gamma': 0.01},
 {'C': 10.0, 'gamma': 0.10000000000000001},
 {'C': 10.0, 'gamma': 1.0},
 {'C': 100.0, 'gamma': 0.0001},
 {'C': 100.0, 'gamma': 0.001},
 {'C': 100.0, 'gamma': 0.01},
 {'C': 100.0, 'gamma': 0.10000000000000001},
 {'C': 100.0, 'gamma': 1.0}]

Let's write a function to load the data from a CV split file and compute the validation score for a given parameter set and model:

In [67]:
def compute_evaluation(cv_split_filename, model, params):
    """Function executed by a worker to evaluate a model on a CV split"""
    # All module imports should be executed in the worker namespace
    from sklearn.externals import joblib

    X_train, y_train, X_validation, y_validation = joblib.load(
        cv_split_filename, mmap_mode='c')
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    validation_score = model.score(X_validation, y_validation)
    return validation_score

In [68]:
def grid_search(lb_view, model, cv_split_filenames, param_grid):
    """Launch all grid search evaluation tasks."""
    all_tasks = []
    all_parameters = list(ParameterGrid(param_grid))
    
    for i, params in enumerate(all_parameters):
        task_for_params = []
        
        for j, cv_split_filename in enumerate(cv_split_filenames):    
            t = lb_view.apply(
                compute_evaluation, cv_split_filename, model, params)
            task_for_params.append(t) 
        
        all_tasks.append(task_for_params)
        
    return all_parameters, all_tasks

Let's try on the digits dataset that we splitted previously as memmapable files:

In [69]:
from sklearn.svm import SVC

lb_view = client.load_balanced_view()
model = SVC()
svc_params = {
    'C': np.logspace(-1, 2, 4),
    'gamma': np.logspace(-4, 0, 5),
}

all_parameters, all_tasks = grid_search(
   lb_view, model, digits_split_filenames, svc_params)

In [70]:
import time
time.sleep(5)

The `grid_search` function is using the asynchronous API of the `LoadBalancedView`, we can hence monitor the progress:

In [71]:
def progress(tasks):
    return np.mean([task.ready() for task_group in tasks
                                 for task in task_group])

In [75]:
print("Tasks completed: {0}%".format(100 * progress(all_tasks)))

Tasks completed: 100.0%


Even better, we can introspect the completed task to find the best parameters set so far:

In [76]:
def find_bests(all_parameters, all_tasks, n_top=5):
    """Compute the mean score of the completed tasks"""
    mean_scores = []
    
    for param, task_group in zip(all_parameters, all_tasks):
        scores = [t.get() for t in task_group if t.ready()]
        if len(scores) == 0:
            continue
        mean_scores.append((np.mean(scores), param))
                   
    return sorted(mean_scores, reverse=True)[:n_top]

In [77]:
from pprint import pprint

print("Tasks completed: {0}%".format(100 * progress(all_tasks)))
pprint(find_bests(all_parameters, all_tasks))

Tasks completed: 100.0%
[(0.99022222222222211, {'C': 1.0, 'gamma': 0.001}),
 (0.98888888888888893, {'C': 100.0, 'gamma': 0.001}),
 (0.98888888888888893, {'C': 10.0, 'gamma': 0.001}),
 (0.98755555555555552, {'C': 10.0, 'gamma': 0.0001}),
 (0.98711111111111127, {'C': 100.0, 'gamma': 0.0001})]


### Optimization Trick: Truncated Randomized Search

It is often wasteful to search all the possible combinations of parameters as done previously, especially if the number of parameters is large (e.g. more than 3).

To speed up the discovery of good parameters combinations, it is often faster to randomized the search order and allocate a budget of evaluations, e.g. 10 or 100 combinations.

See [this JMLR paper by James Bergstra](http://jmlr.csail.mit.edu/papers/v13/bergstra12a.html) for an empirical analysis of the problem. The interested reader should also have a look at [hyperopt](https://github.com/jaberg/hyperopt) that further refines this parameter search method using meta-optimizers.

Randomized Parameter Search has just been implemented in the master branch of scikit-learn be part of the 0.14 release.

---
StarCluster
---

![](https://camo.githubusercontent.com/a878477e72a1dc9fff694d9e5e2d6e743e934732/687474703a2f2f737461722e6d69742e6564752f636c75737465722f646f63732f6c61746573742f5f696d616765732f73636f766572766965772e706e67)

StarCluster comes out of the STAR program at MIT. STAR stands for "Software Tools for Academics and Researchers". It is used to quickly provision a cluster of EC2 instances. Like EMR, it automatically configures them to be used as a cluster (rather than as independent machines) with one controller and many workers. Unlike EMR, it does not have a GUI. Also, Hadoop is just one of many plugins for StarCluster. The most important plugin, however, is IPython Cluster.

<!--
### Brian's Rant

I 💀 the terms "Master" & "Slave" 😡 

They are simply awful and have no place.

Please use "Leader"/"Coordinator" and "Follower" / "Worker"
-->

<br>
<br>

-----